In [1]:
import pandas as pd
import datapackage
import geopy.distance
import time

In [2]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

----

#### Read in reference with location (lat,lon,city,state,country) dataset

In [3]:
df = pd.read_csv('../reference_w_loc.csv')

In [4]:
df.head()

,id,description,duration,location,reported_at,shape,sighted_at,geocoded_latitude,geocoded_longitude,city,state,country
0,0,"Man repts. witnessing &quot;flash, followed by...",NaN,"Iowa City, IA",19951009,unknown,19951009,41.661256,-91.529911,Iowa City,Iowa,us
1,1,"Man on Hwy 43 SW of Milwaukee sees large, bri...",2 min.,"Milwaukee, WI",19951011,unknown,19951010,43.034993,-87.922497,Milwaukee,Wisconsin,us
2,2,Telephoned Report:CA woman visiting daughter w...,NaN,"Shelton, WA",19950103,unknown,19950101,47.215094,-123.100707,Shelton,Washington,us
3,3,Man repts. son&apos;s bizarre sighting of smal...,2 min.,"Columbia, MO",19950510,unknown,19950510,38.951883,-92.333737,Columbia,Missouri,us
4,4,Anonymous caller repts. sighting 4 ufo&apos;s ...,NaN,"Seattle, WA",19950614,unknown,19950611,47.603832,-122.330062,Seattle,Washington,us


----

#### Read in Airport data

In [5]:
data_url = 'https://datahub.io/core/airport-codes/datapackage.json'
package = datapackage.Package(data_url)
resources = package.resources
for resource in resources:
    if resource.tabular:
        airport_df = pd.read_csv(resource.descriptor['path'])
        airport_df = airport_df[airport_df.iso_country == "US"]

In [6]:
airport_df.head(5)

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.949200,-151.695999,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
4,6526,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,NaN,NaN,NaN,NaN,NaN,00AR


In [15]:
airport_df.shape

(22389, 18)

#### unique airport types

In [16]:
airport_df.type.unique()

array(['heliport', 'small_airport', 'closed', 'seaplane_base',
       'balloonport', 'medium_airport', 'large_airport'], dtype=object)

#### There are 172 large airports

In [17]:
airport_df[airport_df.type == 'large_airport'].shape

(172, 18)

In [18]:
large_airport_df = airport_df[airport_df.type == 'large_airport']

#### 687 medium airports

In [19]:
airport_df[airport_df.type == 'medium_airport'].shape

(687, 18)

In [20]:
medium_airport_df = airport_df[airport_df.type == 'medium_airport']

#### 13865 small airports

In [21]:
airport_df[airport_df.type == 'small_airport'].shape

(13865, 18)

In [22]:
small_airport_df = airport_df[airport_df.type == 'small_airport']

#### 858 closed airports -- could be interesting to see if there is any correlation here

In [23]:
airport_df[airport_df.type == 'closed'].shape

(858, 18)

In [24]:
closed_airport_df = airport_df[airport_df.type == 'closed']

----

### haversine distance

In [28]:
# https://stackoverflow.com/questions/43577086/pandas-calculate-haversine-distance-within-each-group-of-rows/43577275#43577275
# https://stackoverflow.com/questions/29545704/fast-haversine-approximation-python-pandas/29546836#29546836
import numpy as np

# haversine distance
def distance_np(c1, c2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1 = c1 
    lon2, lat2 = c2
    
    lon2_,lat2_ = lon2,lat2
    
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    miles = km/1.609344
    return miles

#### differences between different distance metrics
It is clear that vincenty is most precise, but haversine is great for quick computation

In [33]:
# great circle distance
geopy.distance.great_circle((41.6613,-91.5299), (41.680318, -91.598565))

Distance(6.08446133018)

In [34]:
# vincenty distance
geopy.distance.vincenty((41.6613,-91.5299), (41.680318, -91.598565)).miles

3.787775939330663

In [35]:
distance_np((41.6613,-91.5299), (41.680318, -91.598565))

4.7414498663059712

----

### Add distance and name of closest large/medium/small airport from UFO sighting

In [36]:
df['closest_LARGE_airport_name'] = ''
df['closest_LARGE_airport_distance'] = float("inf")

df['closest_MEDIUM_airport_name'] = ''
df['closest_MEDIUM_airport_distance'] = float("inf")

df['closest_SMALL_airport_name'] = ''
df['closest_SMALL_airport_distance'] = float("inf")

#### The following takes ~35 minutes to run -- read the resulting df from pickle (data/df_nearest_airports.pkl) instead of running

In [40]:
t0 = time.time()

large_airport_df = airport_df[airport_df.type == 'large_airport']
large_airport_df['distance_from_target'] = float("inf")

medium_airport_df = airport_df[airport_df.type == 'medium_airport']
medium_airport_df['distance_from_target'] = float("inf")

small_airport_df = airport_df[airport_df.type == 'small_airport']
small_airport_df['distance_from_target'] = float("inf")

closed_airport_df = airport_df[airport_df.type == 'closed']
closed_airport_df['distance_from_target'] = float("inf")

for index,row in df.iterrows():
    longitude = row['geocoded_longitude']
    latitude = row['geocoded_latitude']

    # skip rows without coordinates
    if longitude == "NA" or latitude == "NA":
        continue
    
    large_airport_with_distance_df = large_airport_df.assign(distance_from_target = distance_np((latitude,longitude),(large_airport_df['latitude_deg'],large_airport_df['longitude_deg'])))
    smallest_distance_row = large_airport_with_distance_df.nsmallest(1, 'distance_from_target').head(1).squeeze()
    closest_LARGE_airport_name = smallest_distance_row['name']
    closest_LARGE_airport_distance = smallest_distance_row['distance_from_target']
    df.ix[index, 'closest_LARGE_airport_name'] = closest_LARGE_airport_name
    df.ix[index, 'closest_LARGE_airport_distance'] = closest_LARGE_airport_distance
    
    medium_airport_with_distance_df = medium_airport_df.assign(distance_from_target = distance_np((latitude,longitude),(medium_airport_df['latitude_deg'],medium_airport_df['longitude_deg'])))
    smallest_distance_row = medium_airport_with_distance_df.nsmallest(1, 'distance_from_target').head(1).squeeze()
    closest_MEDIUM_airport_name = smallest_distance_row['name']
    closest_MEDIUM_airport_distance = smallest_distance_row['distance_from_target']
    df.ix[index, 'closest_MEDIUM_airport_name'] = closest_MEDIUM_airport_name
    df.ix[index, 'closest_MEDIUM_airport_distance'] = closest_MEDIUM_airport_distance
    
    small_airport_with_distance_df = small_airport_df.assign(distance_from_target = distance_np((latitude,longitude),(small_airport_df['latitude_deg'],small_airport_df['longitude_deg'])))
    smallest_distance_row = small_airport_with_distance_df.nsmallest(1, 'distance_from_target').head(1).squeeze()
    closest_SMALL_airport_name = smallest_distance_row['name']
    closest_SMALL_airport_distance = smallest_distance_row['distance_from_target']
    df.ix[index, 'closest_SMALL_airport_name'] = closest_SMALL_airport_name
    df.ix[index, 'closest_SMALL_airport_distance'] = closest_SMALL_airport_distance
    
    closed_airport_with_distance_df = closed_airport_df.assign(distance_from_target = distance_np((latitude,longitude),(closed_airport_df['latitude_deg'],closed_airport_df['longitude_deg'])))
    smallest_distance_row = closed_airport_with_distance_df.nsmallest(1, 'distance_from_target').head(1).squeeze()
    closest_CLOSED_airport_name = smallest_distance_row['name']
    closest_CLOSED_airport_distance = smallest_distance_row['distance_from_target']
    df.ix[index, 'closest_CLOSED_airport_name'] = closest_CLOSED_airport_name
    df.ix[index, 'closest_CLOSED_airport_distance'] = closest_CLOSED_airport_distance

t1 = time.time()
print t1-t0

1946.99681997


In [42]:
df.head()

,id,description,duration,location,reported_at,shape,sighted_at,geocoded_latitude,geocoded_longitude,city,state,country,closest_LARGE_airport_name,closest_LARGE_airport_distance,closest_MEDIUM_airport_name,closest_MEDIUM_airport_distance,closest_SMALL_airport_name,closest_SMALL_airport_distance,closest_CLOSED_airport_name,closest_CLOSED_airport_distance
0,0,"Man repts. witnessing &quot;flash, followed by...",NaN,"Iowa City, IA",19951009,unknown,19951009,41.661256,-91.529911,Iowa City,Iowa,us,The Eastern Iowa Airport,12.498008,Chippewa Valley Regional Airport,6.616011,Marion Airport,0.683006,Pete's Patch Airport,3.912630
1,1,"Man on Hwy 43 SW of Milwaukee sees large, bri...",2 min.,"Milwaukee, WI",19951011,unknown,19951010,43.034993,-87.922497,Milwaukee,Wisconsin,us,General Mitchell International Airport,1.801887,Kenosha Regional Airport,1.157715,Herbert C. Maas Airport,1.716202,General Mitchell Intl Heliport,1.552094
2,2,Telephoned Report:CA woman visiting daughter w...,NaN,"Shelton, WA",19950103,unknown,19950101,47.215094,-123.100707,Shelton,Washington,us,McChord Air Force Base,43.233065,Olympia Regional Airport,16.484896,Sanderson Field,3.339607,Teufel's Farm Strip,64.820703
3,3,Man repts. son&apos;s bizarre sighting of smal...,2 min.,"Columbia, MO",19950510,unknown,19950510,38.951883,-92.333737,Columbia,Missouri,us,Bill & Hillary Clinton National Airport/Adams ...,13.834501,Columbia Regional Airport,7.889876,Willhite Airport,3.120229,Iowa Army Natl Guard Heliport,11.244884
4,4,Anonymous caller repts. sighting 4 ufo&apos;s ...,NaN,"Seattle, WA",19950614,unknown,19950611,47.603832,-122.330062,Seattle,Washington,us,Boeing Field King County International Airport,3.343976,Snohomish County (Paine Field) Airport,11.644950,Renton Municipal Airport,8.871067,Blaine Municipal Airport,58.478816


#### save df to pick / read df from pickle

In [43]:
df.to_pickle('../pickle_files/df_nearest_airports.pkl')
# df = pd.read_pickle('../pickle_files/df_nearest_airports.pkl')

##### df at this point has been updated
Added the following features: <br/>
1) closest LARGE airport name & distance<br/>
2) closest MEDIUM airport name & distance<br/>
3) closest SMALL airport name & distance<br/>

In [44]:
df.to_csv('reference_w_airports.csv', index=False)